In [1]:
!pip install pennylane

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.2 MB/s eta 0:00:00


In [2]:
import pennylane as qml
from pennylane import numpy as np

### Task 1 - Pennylane Basics
By reviewing Pennylane Codebook, we summarize different strategies when dealing with basic quantum computing with single qubit and multiple qubit circuits. Particularly:
- Introduction to Quantum Computing
- Single Qubit Gates
- Circuits with Many Qubits

### 1. Introduction to Quantum Computing
We first learn about some basics of quantum computing. Some points include:

#### 1.1 Qubits
All quantum states is a superposition of $|0>$ and $|1>$, which can be expressed as $\alpha|0>+\beta |1>$, where $\alpha, \beta \in C$ they can be complex numbers. And $|\alpha|^2+|\beta|^2=1$. Hence, for all unnormalized vector we can divide both $\alpha$ and $\beta$ by $\sqrt{|\alpha|^2+|\beta|^2}$ to normalize it.

Using Numpy, we can get the square of the absolute value of a complex number by ```np.abs(alpha)**2``` or ```alpha.real**2 + alpha.imag**2```.





In [3]:

ket_0 = np.array([1, 0])
ket_1 = np.array([0, 1])

def normalize_state(alpha, beta):
    """Compute a normalized quantum state given arbitrary amplitudes.

    Args:
        alpha (complex): The amplitude associated with the |0> state.
        beta (complex): The amplitude associated with the |1> state.

    Returns:
        np.array[complex]: A vector (numpy array) with 2 elements that represents
        a normalized quantum state.
    """
    normalizing_factor = np.sqrt(alpha.real**2 + alpha.imag**2 +beta.real**2 + beta.imag**2)
    return alpha/normalizing_factor*ket_0 + beta/normalizing_factor*ket_1


To calculate the **inner product** of two states, we can use ```np.inner()```. However, since we can have complex number in the state, we need to use the conjugate on the first item.

In [4]:
def inner_product(state_1, state_2):
    """Compute the inner product between two states.

    Args:
        state_1 (np.array[complex]): A normalized quantum state vector
        state_2 (np.array[complex]): A second normalized quantum state vector

    Returns:
        complex: The value of the inner product <state_1 | state_2>.
    """

    return np.inner(np.conjugate(state_1), state_2)

When measuring the above state $\alpha|0>+\beta|1>$, the outcome should be a random choice of {0,1} with $Pr(0)=|\alpha|^2$ and $Pr(1)=|\beta|^2$.

And when we are applying a **unitary** operation to the current state we could express as ```U.dot(state)``` or ```np.dot(U, state)```.

In [5]:
U = np.array([[1, 1], [1, -1]]) / np.sqrt(2)

def initialize_state():
    """Prepare a qubit in state |0>.
    Returns:
        np.array[float]: the vector representation of state |0>.
    """
    return np.array([1.0 ,0.0])

def apply_u(state):
    """Apply a quantum operation."""
    return np.dot(U, state)

def measure_state(state, num_meas):
    """Measure a quantum state num_meas times."""
    p_alpha = np.abs(state[0]) ** 2
    p_beta = np.abs(state[1]) ** 2
    meas_outcome = np.random.choice([0, 1], p=[p_alpha, p_beta], size=num_meas)
    return meas_outcome

def quantum_algorithm():
    """Use the functions above to implement the quantum algorithm described above.

    Try and do so using three lines of code or less!

    Returns:
        np.array[int]: the measurement results after running the algorithm 100 times
    """

    # PREPARE THE STATE, APPLY U, THEN TAKE 100 MEASUREMENT SAMPLES
    x = initialize_state()
    x = apply_u(x)
    return measure_state(x, 100)

#### 1.2 Quantum Circuits
In PennyLane, all circuits need to be converted to **QNode** which can be called with the same parameters as the orignial circuit. QNode takes in a *circuit* function and a *device* ```dev```.
There are two ways to convert the circuit function into a QNode.
1. Using ```my_qnode = qml.QNode(my_circuit, dev)```
2. Using the decorator outside of the function:
```
@qml.qnode(dev)
def my_circuit(...):
```

A sample device is defined as follow: ```dev = qml.device("default.qubit", wires=3)``` where 3 is the number of qubits in this case.



If the unitary operation is already specify as a matrix U, then it can be applied as follow: ```qml.QubitUnitary(U, wires=wire)```.

In [6]:
dev = qml.device("default.qubit", wires=1)
U = np.array([[1, 1], [1, -1]]) / np.sqrt(2)

@qml.qnode(dev)
def my_circuit():
    qml.QubitUnitary(U, wires=0)
    return qml.state()

my_circuit()

tensor([0.70710678+0.j, 0.70710678+0.j], requires_grad=True)

### 2. Single Qubit Gates
#### 2.1 Basics
Some popular single-qubit gate operations are:
- RX, RY, RZ rotations: ```qml.RX(angle, wires=wire)``` or ```qml.RY()```, ```qml.RZ()```, which specify the angle of rotation in the X, Y, Z axis and the index of qubit to act on.
- Hadamard gate to create superposition ```qml.Hadamard(wires=wire)```
- Pauli X operations: usually used at the beginning of the circuit to change state $|0>$ to state $|1>$. In PennyLane, ```qml.PauliX(wires=wire)```
- Pauli Y: ```qml.Y(wires=wire)```
- Pauli Z operation does not change $|0>$, but change the phase of $|1>$ to $-|1>$. It can be expressed as ```qml.Z(wires=wire)``` or using RZ rotation ```qml.RZ(np.pi, wires=wire)```.
- T, S operations, where $T=RZ(\pi/4)$ and $S=RZ(\pi/2)$.
- Parameterized rotation operation ```qml.Rot(phi, theta, omega, wires=wire)```


In [7]:
dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def apply_hxh(state):
    if state:
        qml.PauliX(wires=0)
    qml.Hadamard(wires=0)
    qml.PauliX(wires=0)
    qml.Hadamard(wires=0)
    return qml.state()

print(apply_hxh(0))
print(apply_hxh(1))


[1.+0.j 0.+0.j]
[ 0.+0.j -1.+0.j]


The adjoint of any operation can be expressed as ```qml.adjoint(qml.RX)(angle, wires=wire)``` where ```qml.RX``` can be switched into any operation followed by the same arguments as the original operation

In [8]:
dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def circuit():
    """Use RZ to produce the same action as Pauli Z on the |+> state.
    Then apply the adjoint of T gate
    Returns:
        np.array[complex]: The state of the qubit after the operations.
    """
    # CREATE THE |+> STATE
    qml.Hadamard(wires=0)
    qml.RZ(np.pi, wires=0)
    qml.adjoint(qml.T)(wires=0)
    return qml.state()

#### 2.2 Universal Gate Sets
The most general expression of single-qubit unitary operation is:
$U(\phi,\theta,\omega)=\pmatrix{e^{-i(\phi+\omega)/2}cos(\theta/2) & -e^{i(\phi+\omega)/2}sin(\theta/2) \\ e^{-i(\phi+\omega)/2}sin(\theta/2) & e^{i(\phi+\omega)/2}cos(\theta/2)}$

In PennyLane, $U$ can be defined by ```qml.Rot(phi, theta, omega, wires=wire)```, which is equivalent to applying the following gates in sequence:
```
qml.RZ(phi, wires=0)
qml.RY(theta, wires=0)
qml.RZ(omega, wires=0)
```

Note that applying two rotations of the same type is equivalent to applying one rotation with the cumulative angle.

By using the above setting, we can find that $H = RZ(\pi/2)RX(\pi/2)RZ(\pi/2)$.

In [9]:
dev = qml.device("default.qubit", wires=1)
phi, theta, omega = np.pi/2 , np.pi/2, np.pi/2

@qml.qnode(dev)
def hadamard_with_rz_rx():
    qml.RZ(phi, wires=0)
    qml.RX(theta, wires=0)
    qml.RZ(omega, wires=0)
    return qml.state()

Simiarly, $H$ and $T$ are also a universal gate set. By combining just these two gates, we can approximate any single-qubit operation.

#### 2.2 State Preparation
Given an arbitary state vector, we can use ```qml.MottonenStatePreparation``` to prepare the state.

In [10]:
v = np.array([0.52889389 - 0.14956775j, 0.67262317 + 0.49545818j])

dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def prepare_state(state=v):
    qml.MottonenStatePreparation(v, wires=0)
    return qml.state()


print(prepare_state(v))
print()
print(qml.draw(prepare_state, expansion_strategy="device")(v))

[0.52889389-0.14956775j 0.67262317+0.49545818j]

0: ──RY(1.98)──RZ(0.91)──GlobalPhase(-0.18)─┤  State


#### 2.3 Measuremnt
There are a few ways to measure the circuit after the gate operations
- Show the state vector after the operation ```qml.state()```
- Show the probability of being in each state ```qml.probs(wires=wires)```
- Measure the probabilities of being in each state, where the states could be different basis, e.g. Y basis. It could be done by using ```qml.adjoint(basis_rotation)()``` after preparing the states.
- Measure the expected value of different observables using ```qml.expval(qml.PauliY(wires=0))```, which can be replaced by Pauli X and Pauli Z.
- Access the samples directly using ```qml.sample(qml.PauliY(wires=0))```

In [11]:
dev = qml.device("default.qubit", wires=1)

def prepare_psi():
    qml.RX(np.pi*2/3,wires=0)
    qml.Z(wires=0)

# WRITE A QUANTUM FUNCTION THAT SENDS BOTH |0> TO |y_+> and |1> TO |y_->
def y_basis_rotation():
    qml.Hadamard(wires=0)
    qml.S(wires=0)

@qml.qnode(dev)
def measure_in_y_basis():
    # PREPARE THE STATE
    prepare_psi()

    qml.adjoint(y_basis_rotation)()

    return qml.probs(wires=0)


print(measure_in_y_basis())


[0.9330127 0.0669873]


In [12]:
shot_results = []

# Different numbers of shots
shot_values = [100, 1000, 10000, 100000, 1000000]

def circuit():
    qml.RX(np.pi/4, wires=0)
    qml.Hadamard(wires=0)
    qml.Z(wires=0)
    return qml.expval(qml.PauliY(wires=0))

for shots in shot_values:
    # CREATE A DEVICE, CREATE A QNODE, AND RUN IT
    dev = qml.device('default.qubit', wires=1, shots=shots)
    qnode = qml.QNode(circuit, dev)
    # STORE RESULT IN SHOT_RESULTS ARRAY
    shot_results.append(qnode())

print(qml.math.unwrap(shot_results))

[-0.64, -0.732, -0.7106, -0.70874, -0.70688]


### 3. Multi-Qubit Circuits
#### 3.1 Prepare basis states based on state ID
We can use ```np.binary_repr(int)``` to translate the integer into binary strings. And then directly using ```qml.BasisEmbedding()``` to convert the bitstring into circuits.

In [13]:
num_wires = 3
dev = qml.device("default.qubit", wires=num_wires)

@qml.qnode(dev)
def make_basis_state(basis_id):
    """Produce the 3-qubit basis state corresponding to |basis_id>.
    Note that the system starts in |000>.
    Args:
        basis_id (int): An integer value identifying the basis state to construct.
    Returns:
        np.array[complex]: The computational basis state |basis_id>.
    """
    b = np.binary_repr(basis_id)
    print(b)
    if len(b)<3:
        b = '0'*(3-len(b))+b
    qml.BasisEmbedding(features=[int(i) for i in b], wires=range(3))
    return qml.state()

basis_id = 3
print(f"Output state = {make_basis_state(basis_id)}")

11
Output state = [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


#### 3.2 Measurement
In a multi-qubit circuit, we can either measure the different qubits separately, or directly measure a multi-qubit observable.
- Separate measurements can be combined in a tuple.
- Two-qubit observable measurement can be expressed as ```qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))```


In [15]:
dev = qml.device("default.qubit", wires=2)
@qml.qnode(dev)
def circuit_1(theta):
    """Implement the circuit and measure Z I and I Z.
    Args:
        theta (float): a rotation angle.
    Returns:
        float, float: The expectation values of the observables Z I, and I Z
    """
    qml.RX(theta, wires=0)
    qml.RY(2*theta, wires=1)
    return (qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1)))

@qml.qnode(dev)
def circuit_2(theta):
    """Implement the circuit and measure Z Z.
    Args:
        theta (float): a rotation angle.
    Returns:
        float: The expectation value of the observable Z Z
    """
    qml.RX(theta, wires=0)
    qml.RY(2*theta, wires=1)

    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

def zi_iz_combination(ZI_results, IZ_results):
    """Implement a function that acts on the ZI and IZ results to
    produce the ZZ results. How do you think they should combine?
    Args:
        ZI_results (np.array[float]): Results from the expectation value of
            ZI in circuit_1.
        IZ_results (np.array[float]): Results from the expectation value of
            IZ in circuit_2.
    Returns:
        np.array[float]: A combination of ZI_results and IZ_results that
        produces results equivalent to measuring ZZ.
    """
    combined_results = np.zeros(len(ZI_results))
    combined_results = ZI_results*IZ_results
    return combined_results

theta = np.linspace(0, 2 * np.pi, 100)
# Run circuit 1, and process the results
circuit_1_results = np.array([circuit_1(t) for t in theta])

ZI_results = circuit_1_results[:, 0]
IZ_results = circuit_1_results[:, 1]
combined_results = zi_iz_combination(ZI_results, IZ_results)

ZZ_results = np.array([circuit_2(t) for t in theta])

#### 3.3 Entanglement
Entanglements are added by controlled gate operations and other gates like SWAP.
- Controlled NOT: ```qml.CNOT(wires=[control, target])```
- Controlled rotation: ```qml.CRX(theta, wires=[control, target])``` or ```qml.CRY``` and ```qml.CRZ```.
- SWAP gate: ```qml.SWAP([qubit0,qubit1])```
- Toffoli gate, which is controlled-controlled-NOT gate: ```qml.Toffoli(wires=[control1, control2, target])```
- Multi-controlled gate: ```qml.MultiControlledX(control_wires=[0, 1, 2, 3], wires=4, control_values="1011")```

Similarly, we can use CNOT to generate SWAP gate, and using Toffoli to generate controlled SWAP.

In [16]:
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def apply_swap():
    qml.SWAP([0,1])
    return qml.state()

@qml.qnode(dev)
def apply_swap_with_cnots():
    # IMPLEMENT THE SWAP GATE USING A SEQUENCE OF CNOTS
    qml.CNOT([0,1])
    qml.CNOT([1,0])
    qml.CNOT([0,1])
    return qml.state()

print(f"Regular SWAP state = {apply_swap()}")
print(f"CNOT SWAP state = {apply_swap_with_cnots()}")

Regular SWAP state = [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
CNOT SWAP state = [1.+0.j 0.+0.j 0.+0.j 0.+0.j]


In [17]:
dev = qml.device("default.qubit", wires=3)

@qml.qnode(dev)
def no_swap():
    return qml.state()

@qml.qnode(dev)
def controlled_swap():
    # PERFORM A CONTROLLED SWAP USING A SEQUENCE OF TOFFOLIS
    qml.Toffoli(wires=[0, 1, 2])
    qml.Toffoli(wires=[0, 2, 1])
    qml.Toffoli(wires=[0, 1, 2])
    return qml.state()

print(no_swap())
print(controlled_swap())

[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


#### 3.4 Bell States

In [18]:
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def prepare_psi_plus():
    # PREPARE (1/sqrt(2)) (|00> + |11>)
    qml.Hadamard(0)
    qml.CNOT([0,1])
    return qml.state()

@qml.qnode(dev)
def prepare_psi_minus():
    # PREPARE (1/sqrt(2)) (|00> - |11>)
    qml.X(0)
    qml.Hadamard(0)
    qml.CNOT([0,1])
    return qml.state()

@qml.qnode(dev)
def prepare_phi_plus():
    # PREPARE  (1/sqrt(2)) (|01> + |10>)
    qml.Hadamard(0)
    qml.X(1)
    qml.CNOT([0,1])
    return qml.state()

@qml.qnode(dev)
def prepare_phi_minus():
    # PREPARE  (1/sqrt(2)) (|01> - |10>)
    qml.X(0)
    qml.Hadamard(0)
    qml.X(1)
    qml.CNOT([0,1])
    return qml.state()

psi_plus = prepare_psi_plus()
psi_minus = prepare_psi_minus()
phi_plus = prepare_phi_plus()
phi_minus = prepare_phi_minus()

# Uncomment to print results
print(f"|ψ_+> = {psi_plus}")
print(f"|ψ_-> = {psi_minus}")
print(f"|ϕ_+> = {phi_plus}")
print(f"|ϕ_-> = {phi_minus}")

|ψ_+> = [0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]
|ψ_-> = [ 0.70710678+0.j  0.        +0.j  0.        +0.j -0.70710678+0.j]
|ϕ_+> = [0.        +0.j 0.70710678+0.j 0.70710678+0.j 0.        +0.j]
|ϕ_-> = [ 0.        +0.j  0.70710678+0.j -0.70710678+0.j  0.        +0.j]
